### Taylor Green convergence test

Convergence as function of the spatial resolution, using the Taylor Green vortex.

Taylor, G. I., & Green, A. E. (1937). Mechanism of the production of small eddies from large ones. Proceedings of the Royal Society of London. Series A-Mathematical and Physical Sciences, 158(895), 499-521.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import microhh_tools as mht
from taylorgreen_funcs import Parse_TaylorGreen

In [2]:
"""
Read/parse cases.
"""
ns = np.array([16, 32, 64, 128, 256])
dxs = 1. / ns

time = 1
visc = (8. * np.pi**2. * 100.)**(-1.)

def parse(name):
    cases = []
    err_u = []
    err_w = []
    err_p = []
    for n in ns:
        c = Parse_TaylorGreen(time, visc, f'local_itot{n:03d}_{name}')
        err_u.append(c.u_err)
        err_w.append(c.w_err)
        err_p.append(c.p_err)
        cases.append(c)

    return cases, err_u, err_w, err_p

# 2nd order.
advec2,  err_u_2,  err_w_2,  err_p_2  = parse('swadvec2')

## 4th order with `advec4m`.
advec4m, err_u_4m, err_w_4m, err_p_4m = parse('swadvec4m')

## 4th order with `advec4`.
advec4,  err_u_4,  err_w_4,  err_p_4  = parse('swadvec4')

In [ ]:
"""
Plot convergence schemes.
"""
plt.close('all')

"""
Plot convergence schemes.
"""
slope2 = 0.01 * (dxs[:] / dxs[0])**2.
slope41 = 0.002 * (dxs[:] / dxs[0])**4.
slope42 = 0.000001 * (dxs[:] / dxs[0])**4.

plt.figure(layout='tight')

plt.loglog(dxs, err_u_2, 'o-', color='C0', label='u (2)')
plt.loglog(dxs, err_w_2, 'v-', color='C0', label='w (2')
plt.loglog(dxs, err_p_2, '^-', color='C0', label='p (2')

plt.loglog(dxs, err_u_4m, 'o--', color='C1', label='u (4m)')
plt.loglog(dxs, err_w_4m, 'v--', color='C1', label='w (4m)')
plt.loglog(dxs, err_p_4m, '^--', color='C1', label='p (4m)')

plt.loglog(dxs, err_u_4, 'o--', color='C2', label='u (4)')
plt.loglog(dxs, err_w_4, 'v--', color='C2', label='w (4)')
plt.loglog(dxs, err_p_4, '^--', color='C2', label='p (4)')

plt.loglog(dxs, slope2, 'k--', label=r'$\mathcal{O}(2)$')
plt.loglog(dxs, slope41, 'k-.',  label=r'$\mathcal{O}(4)$')
plt.loglog(dxs, slope42, 'k-.')

plt.xlabel(r'$\Delta$x')
plt.ylabel('Error')
plt.legend(ncol=4)

plt.savefig('taylorgreen_convergence.png')

In [ ]:
"""
Plot cross-sections with errors of highest resolution case.
"""
def plot_cross_error(case, var, order, sp):
    """
    Plot cross-section of 2nd and 4th order error for variable `var`.
    """

    plt.subplot(sp, aspect='equal')
    plt.title(f'Error {var} {order}')
    plt.pcolormesh(case.x, case.z, getattr(case, var)[:, 0, :] - getattr(case, f'{var}_ref')[:, 0, :], rasterized=True)
    plt.xlim(min(case.xh), max(case.xh))
    plt.ylim(min(case.z), max(case.z))
    plt.xlabel('x')
    plt.ylabel('z')
    plt.colorbar(shrink=0.6)


plt.figure(figsize=(9,7), layout='tight')

plot_cross_error(advec2[-1], 'u', '2nd', 321)
plot_cross_error(advec4[-1], 'u', '4th', 322)

plot_cross_error(advec2[-1], 'w', '2nd', 323)
plot_cross_error(advec4[-1], 'w', '4th', 324)

plot_cross_error(advec2[-1], 'p', '2nd', 325)
plot_cross_error(advec4[-1], 'p', '4th', 326)

plt.savefig('taylorgreen_errors.png')